**构造xgboost模型，评估数据集难度**

In [1]:
import os
import sys
BASE_DIR = r"C:\Users\nino.zhou\Desktop\my_project\cp_newbank_mobile_behavior"
sys.path.insert(0, BASE_DIR)

import pandas as pd
import numpy as np
from config import setting
from config import setting
import pandas as pd
from xgboost import XGBClassifier
import itertools
from sklearn.externals import joblib
import datetime

C:\Users\nino.zhou\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
df_train = pd.read_csv(setting.BASE_DIR + "\data_model\data\sensor_train.csv")
df_train

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0
...,...,...,...,...,...,...,...,...,...
425354,7291,4561,-0.1,-0.5,-1.7,-0.2,3.4,7.4,18
425355,7291,4647,0.7,-1.9,1.1,0.5,1.7,10.7,18
425356,7291,4735,1.3,-1.4,-0.5,1.4,2.1,8.0,18
425357,7291,4830,1.2,-1.6,-0.1,1.6,1.8,9.2,18


# 1. 特征构造

## 1.1构造是否fragment起点终点的特征

In [3]:
start_mark_list = []
end_mark_list = []
line_num_list = []
last_fragment_id = -1
line_num = 1
for fragment_id in df_train["fragment_id"]:
    if last_fragment_id != fragment_id:
        start_mark_list.append(1)
        end_mark_list.append(1)
        line_num = 1
        line_num_list.append(line_num)
    else:
        start_mark_list.append(0)
        end_mark_list.append(0)
        line_num += 1
        line_num_list.append(line_num)
        
    
    last_fragment_id = fragment_id
end_mark_list.append(1)
end_mark_list = end_mark_list[1:]

In [4]:
len(start_mark_list), len(end_mark_list), len(line_num_list)

(425359, 425359, 425359)

In [5]:
df_train["fragment_start_mark"] = start_mark_list
df_train["fragment_end_mark"] = end_mark_list
df_train["fragment_line_num"] = line_num_list
df_train[50:100]

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,fragment_start_mark,fragment_end_mark,fragment_line_num
50,0,4421,0.0,-0.0,-0.0,0.7,4.7,8.4,0,0,0,51
51,0,4505,-0.0,0.0,-0.1,0.7,4.7,8.5,0,0,0,52
52,0,4596,-0.0,0.1,0.2,0.6,4.7,8.8,0,0,0,53
53,0,4681,-0.1,0.0,-0.0,0.7,4.6,8.5,0,0,0,54
54,0,4768,-0.2,-0.1,0.2,0.9,4.8,8.6,0,0,0,55
55,0,4853,-0.1,-0.1,0.0,0.8,4.7,7.9,0,0,0,56
56,0,4951,0.3,0.1,0.1,0.8,4.7,8.6,0,0,1,57
57,1,58,-0.1,-0.0,0.3,0.8,5.1,8.9,0,1,0,1
58,1,146,0.1,-0.0,0.0,0.8,5.0,8.4,0,0,0,2
59,1,233,0.0,0.1,0.3,0.9,4.9,8.3,0,0,0,3


In [6]:
df_train[["fragment_id", "fragment_start_mark", "fragment_end_mark", "fragment_line_num"]][-100:-50]

,fragment_id,fragment_start_mark,fragment_end_mark,fragment_line_num
425259,7290,0,0,17
425260,7290,0,0,18
425261,7290,0,0,19
425262,7290,0,0,20
425263,7290,0,0,21
425264,7290,0,0,22
425265,7290,0,0,23
425266,7290,0,0,24
425267,7290,0,0,25
425268,7290,0,0,26


## 1.2构造time_point差分特征

In [7]:
df_train["time_point_diff"] = df_train["time_point"].diff().fillna(0)
df_train

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,fragment_start_mark,fragment_end_mark,fragment_line_num,time_point_diff
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0,1,0,1,0.0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0,0,0,2,81.0
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0,0,0,3,90.0
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0,0,0,4,99.0
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0,0,0,5,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425354,7291,4561,-0.1,-0.5,-1.7,-0.2,3.4,7.4,18,0,0,54,84.0
425355,7291,4647,0.7,-1.9,1.1,0.5,1.7,10.7,18,0,0,55,86.0
425356,7291,4735,1.3,-1.4,-0.5,1.4,2.1,8.0,18,0,0,56,88.0
425357,7291,4830,1.2,-1.6,-0.1,1.6,1.8,9.2,18,0,0,57,95.0


# 2. 特征衍生

## 2.1 加减乘除

In [8]:
to_derive_column_list = [column for column in df_train.columns if column not in ["fragment_id", "time_point", "behavior_id"]]
to_derive_column_list

['acc_x',
 'acc_y',
 'acc_z',
 'acc_xg',
 'acc_yg',
 'acc_zg',
 'fragment_start_mark',
 'fragment_end_mark',
 'fragment_line_num',
 'time_point_diff']

In [9]:
import itertools
column_sets = itertools.product(to_derive_column_list, to_derive_column_list)
for col1, col2 in column_sets:
    print(col1, col2)
    df_train[col1 + ":+:" + col2] = df_train[col1] + df_train[col2]
    df_train[col1 + ":-:" + col2] = df_train[col1] - df_train[col2]
    df_train[col1 + ":*:" + col2] = df_train[col1] * df_train[col2]
    df_train[col1 + ":/:" + col2] = df_train[col1] / (df_train[col2] + 10)

acc_x acc_x
acc_x acc_y
acc_x acc_z
acc_x acc_xg
acc_x acc_yg
acc_x acc_zg
acc_x fragment_start_mark
acc_x fragment_end_mark
acc_x fragment_line_num
acc_x time_point_diff
acc_y acc_x
acc_y acc_y
acc_y acc_z
acc_y acc_xg
acc_y acc_yg
acc_y acc_zg
acc_y fragment_start_mark
acc_y fragment_end_mark
acc_y fragment_line_num
acc_y time_point_diff
acc_z acc_x
acc_z acc_y
acc_z acc_z
acc_z acc_xg
acc_z acc_yg
acc_z acc_zg
acc_z fragment_start_mark
acc_z fragment_end_mark
acc_z fragment_line_num
acc_z time_point_diff
acc_xg acc_x
acc_xg acc_y
acc_xg acc_z
acc_xg acc_xg
acc_xg acc_yg
acc_xg acc_zg
acc_xg fragment_start_mark
acc_xg fragment_end_mark
acc_xg fragment_line_num
acc_xg time_point_diff
acc_yg acc_x
acc_yg acc_y
acc_yg acc_z
acc_yg acc_xg
acc_yg acc_yg
acc_yg acc_zg
acc_yg fragment_start_mark
acc_yg fragment_end_mark
acc_yg fragment_line_num
acc_yg time_point_diff
acc_zg acc_x
acc_zg acc_y
acc_zg acc_z
acc_zg acc_xg
acc_zg acc_yg
acc_zg acc_zg
acc_zg fragment_start_mark
acc_zg fragment_e

In [10]:
df_train

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,fragment_start_mark,...,time_point_diff:*:fragment_end_mark,time_point_diff:/:fragment_end_mark,time_point_diff:+:fragment_line_num,time_point_diff:-:fragment_line_num,time_point_diff:*:fragment_line_num,time_point_diff:/:fragment_line_num,time_point_diff:+:time_point_diff,time_point_diff:-:time_point_diff,time_point_diff:*:time_point_diff,time_point_diff:/:time_point_diff
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0,1,...,0.0,0.000000,1.0,-1.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0,0,...,0.0,8.100000,83.0,79.0,162.0,6.750000,162.0,0.0,6561.0,0.890110
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0,0,...,0.0,9.000000,93.0,87.0,270.0,6.923077,180.0,0.0,8100.0,0.900000
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0,0,...,0.0,9.900000,103.0,95.0,396.0,7.071429,198.0,0.0,9801.0,0.908257
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0,0,...,0.0,9.100000,96.0,86.0,455.0,6.066667,182.0,0.0,8281.0,0.900990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425354,7291,4561,-0.1,-0.5,-1.7,-0.2,3.4,7.4,18,0,...,0.0,8.400000,138.0,30.0,4536.0,1.312500,168.0,0.0,7056.0,0.893617
425355,7291,4647,0.7,-1.9,1.1,0.5,1.7,10.7,18,0,...,0.0,8.600000,141.0,31.0,4730.0,1.323077,172.0,0.0,7396.0,0.895833
425356,7291,4735,1.3,-1.4,-0.5,1.4,2.1,8.0,18,0,...,0.0,8.800000,144.0,32.0,4928.0,1.333333,176.0,0.0,7744.0,0.897959
425357,7291,4830,1.2,-1.6,-0.1,1.6,1.8,9.2,18,0,...,0.0,9.500000,152.0,38.0,5415.0,1.417910,190.0,0.0,9025.0,0.904762


## 2.2 均线

In [11]:
def create_mean_feature(df_groupby, col, day):
    return df_groupby[col].rolling(day).mean()

for col in to_derive_column_list:
    for day in [2,4,6,8,10]:
        df_train[col + "_{}_mean".format(day)] = df_train[["fragment_id", col]].groupby("fragment_id").apply(create_mean_feature, col, day).reset_index(drop=True)

In [12]:
df_train = df_train.fillna(0)
df_train.shape

(425359, 463)

# 切分训练验证测试集

In [13]:
def random_train_test_split(df_train):
    from sklearn.model_selection import train_test_split
    X = df_train.drop("behavior_id", axis=1)
    y = df_train["behavior_id"]
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.4, random_state=10086)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=10087)
    return X_train, X_val, y_train, y_val, X_test, y_test
X_train, X_val, y_train, y_val, X_test, y_test = random_train_test_split(df_train)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape, y_test.shape

((204172, 462), (51043, 462), (204172,), (51043,), (170144, 462), (170144,))

In [14]:
def ramdom_df(df):
    """随机打乱df数据的顺序"""
    random_list = list(df.index)
    np.random.shuffle(random_list)
    return df.loc[random_list]

In [15]:
def fragment_random_train_test_split(df_train):
    """按照fragment_id作为基础单元来切分数据集"""
    from sklearn.model_selection import train_test_split
    unique_fragment_id = list(df_train["fragment_id"].drop_duplicates())
    X_train_val_ids, X_test_ids = train_test_split(unique_fragment_id, test_size=0.4, random_state=10086)
    X_train_ids, X_val_ids = train_test_split(X_train_val_ids, test_size=0.2, random_state=10087)
    
    df_train_dataset = df_train[df_train["fragment_id"].apply(lambda x: True if x in X_train_ids else False)]
    df_train_dataset = df_train_dataset.drop("fragment_id", axis=1)
    df_train_dataset = ramdom_df(df_train_dataset)
    X_train = df_train_dataset.drop("behavior_id", axis=1)
    y_train = df_train_dataset["behavior_id"]
    
    df_val_dataset = df_train[df_train["fragment_id"].apply(lambda x:  True if x in X_val_ids else False)]
    df_val_dataset = df_val_dataset.drop("fragment_id", axis=1)
    df_val_dataset = ramdom_df(df_val_dataset)
    X_val = df_val_dataset.drop("behavior_id", axis=1)
    y_val = df_val_dataset["behavior_id"]
    
    df_test_dataset = df_train[df_train["fragment_id"].apply(lambda x:  True if x in X_test_ids else False)]
    df_test_dataset = df_test_dataset.drop("fragment_id", axis=1)
    df_test_dataset = ramdom_df(df_test_dataset)
    X_test = df_test_dataset.drop("behavior_id", axis=1)
    y_test = df_test_dataset["behavior_id"]

    return X_train, X_val, y_train, y_val, X_test, y_test
X_train, X_val, y_train, y_val, X_test, y_test = fragment_random_train_test_split(df_train)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape, y_test.shape

((204177, 461), (51010, 461), (204177,), (51010,), (170172, 461), (170172,))

# 模型构造与训练

In [17]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(objective="multi:softmax", num_class=19, nthread=-1, eval_metric= 'merror')

param_grid_dict = {
    "n_estimators": [50, 100, 200, 500, 1000],
}
grid = GridSearchCV(xgb_clf, param_grid=param_grid_dict, cv=5)
grid.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=10)

[0]	validation_0-merror:0.50853	validation_1-merror:0.54940
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.47253	validation_1-merror:0.51992
[2]	validation_0-merror:0.45688	validation_1-merror:0.51174
[3]	validation_0-merror:0.44457	validation_1-merror:0.50029
[4]	validation_0-merror:0.43396	validation_1-merror:0.49043
[5]	validation_0-merror:0.42551	validation_1-merror:0.48475
[6]	validation_0-merror:0.41904	validation_1-merror:0.48006
[7]	validation_0-merror:0.41089	validation_1-merror:0.47301
[8]	validation_0-merror:0.40409	validation_1-merror:0.46618
[9]	validation_0-merror:0.39766	validation_1-merror:0.46214
[10]	validation_0-merror:0.39115	validation_1-merror:0.45736
[11]	validation_0-merror:0.38608	validation_1-merror:0.45283
[12]	validation_0-merror:0.38039	validation_1-merror:0.45030
[13]	validation_0-merror:0.37500	validation_1-merror:0

[27]	validation_0-merror:0.31240	validation_1-merror:0.40847
[28]	validation_0-merror:0.30906	validation_1-merror:0.40702
[29]	validation_0-merror:0.30605	validation_1-merror:0.40480
[30]	validation_0-merror:0.30273	validation_1-merror:0.40269
[31]	validation_0-merror:0.29844	validation_1-merror:0.40075
[32]	validation_0-merror:0.29544	validation_1-merror:0.39984
[33]	validation_0-merror:0.29209	validation_1-merror:0.39784
[34]	validation_0-merror:0.28996	validation_1-merror:0.39706
[35]	validation_0-merror:0.28685	validation_1-merror:0.39520
[36]	validation_0-merror:0.28355	validation_1-merror:0.39522
[37]	validation_0-merror:0.28025	validation_1-merror:0.39477
[38]	validation_0-merror:0.27787	validation_1-merror:0.39316
[39]	validation_0-merror:0.27501	validation_1-merror:0.39210
[40]	validation_0-merror:0.27170	validation_1-merror:0.39081
[41]	validation_0-merror:0.26939	validation_1-merror:0.39008
[42]	validation_0-merror:0.26670	validation_1-merror:0.38981
[43]	validation_0-merror

[4]	validation_0-merror:0.43396	validation_1-merror:0.49043
[5]	validation_0-merror:0.42551	validation_1-merror:0.48475
[6]	validation_0-merror:0.41904	validation_1-merror:0.48006
[7]	validation_0-merror:0.41089	validation_1-merror:0.47301
[8]	validation_0-merror:0.40409	validation_1-merror:0.46618
[9]	validation_0-merror:0.39766	validation_1-merror:0.46214
[10]	validation_0-merror:0.39115	validation_1-merror:0.45736
[11]	validation_0-merror:0.38608	validation_1-merror:0.45283
[12]	validation_0-merror:0.38039	validation_1-merror:0.45030
[13]	validation_0-merror:0.37500	validation_1-merror:0.44534
[14]	validation_0-merror:0.36976	validation_1-merror:0.44291
[15]	validation_0-merror:0.36415	validation_1-merror:0.43938
[16]	validation_0-merror:0.35928	validation_1-merror:0.43680
[17]	validation_0-merror:0.35506	validation_1-merror:0.43368
[18]	validation_0-merror:0.34960	validation_1-merror:0.43095
[19]	validation_0-merror:0.34563	validation_1-merror:0.42846
[20]	validation_0-merror:0.340

[36]	validation_0-merror:0.28527	validation_1-merror:0.39816
[37]	validation_0-merror:0.28219	validation_1-merror:0.39720
[38]	validation_0-merror:0.27913	validation_1-merror:0.39649
[39]	validation_0-merror:0.27686	validation_1-merror:0.39602
[40]	validation_0-merror:0.27399	validation_1-merror:0.39486
[41]	validation_0-merror:0.27139	validation_1-merror:0.39406
[42]	validation_0-merror:0.26838	validation_1-merror:0.39335
[43]	validation_0-merror:0.26608	validation_1-merror:0.39233
[44]	validation_0-merror:0.26406	validation_1-merror:0.39108
[45]	validation_0-merror:0.26128	validation_1-merror:0.38988
[46]	validation_0-merror:0.25855	validation_1-merror:0.38900
[47]	validation_0-merror:0.25575	validation_1-merror:0.38800
[48]	validation_0-merror:0.25247	validation_1-merror:0.38730
[49]	validation_0-merror:0.25019	validation_1-merror:0.38687
[50]	validation_0-merror:0.24792	validation_1-merror:0.38567
[51]	validation_0-merror:0.24566	validation_1-merror:0.38453
[52]	validation_0-merror

[68]	validation_0-merror:0.21281	validation_1-merror:0.37463
[69]	validation_0-merror:0.21190	validation_1-merror:0.37479
[70]	validation_0-merror:0.21037	validation_1-merror:0.37418
[71]	validation_0-merror:0.20908	validation_1-merror:0.37357
[72]	validation_0-merror:0.20710	validation_1-merror:0.37355
[73]	validation_0-merror:0.20550	validation_1-merror:0.37263
[74]	validation_0-merror:0.20389	validation_1-merror:0.37218
[75]	validation_0-merror:0.20218	validation_1-merror:0.37197
[76]	validation_0-merror:0.20089	validation_1-merror:0.37134
[77]	validation_0-merror:0.19957	validation_1-merror:0.37146
[78]	validation_0-merror:0.19789	validation_1-merror:0.37126
[79]	validation_0-merror:0.19632	validation_1-merror:0.37157
[80]	validation_0-merror:0.19521	validation_1-merror:0.37097
[81]	validation_0-merror:0.19417	validation_1-merror:0.37097
[82]	validation_0-merror:0.19256	validation_1-merror:0.37057
[83]	validation_0-merror:0.19146	validation_1-merror:0.37083
[84]	validation_0-merror

...

[151]	validation_0-merror:0.11695	validation_1-merror:0.36526
[152]	validation_0-merror:0.11609	validation_1-merror:0.36505
[153]	validation_0-merror:0.11532	validation_1-merror:0.36481
[154]	validation_0-merror:0.11443	validation_1-merror:0.36434
[155]	validation_0-merror:0.11387	validation_1-merror:0.36420
[156]	validation_0-merror:0.11306	validation_1-merror:0.36432
[157]	validation_0-merror:0.11247	validation_1-merror:0.36416
[158]	validation_0-merror:0.11183	validation_1-merror:0.36450
[159]	validation_0-merror:0.11109	validation_1-merror:0.36408
[160]	validation_0-merror:0.11058	validation_1-merror:0.36407
[161]	validation_0-merror:0.11001	validation_1-merror:0.36354
[162]	validation_0-merror:0.10904	validation_1-merror:0.36285
[163]	validation_0-merror:0.10838	validation_1-merror:0.36301
[164]	validation_0-merror:0.10759	validation_1-merror:0.36346
[165]	validation_0-merror:0.10662	validation_1-merror:0.36305
[166]	validation_0-merror:0.10590	validation_1-merror:0.36293
[167]	va

[83]	validation_0-merror:0.19016	validation_1-merror:0.37334
[84]	validation_0-merror:0.18871	validation_1-merror:0.37310
[85]	validation_0-merror:0.18740	validation_1-merror:0.37281
[86]	validation_0-merror:0.18651	validation_1-merror:0.37277
[87]	validation_0-merror:0.18501	validation_1-merror:0.37232
[88]	validation_0-merror:0.18398	validation_1-merror:0.37157
[89]	validation_0-merror:0.18288	validation_1-merror:0.37138
[90]	validation_0-merror:0.18199	validation_1-merror:0.37095
[91]	validation_0-merror:0.18049	validation_1-merror:0.37124
[92]	validation_0-merror:0.17977	validation_1-merror:0.37144
[93]	validation_0-merror:0.17844	validation_1-merror:0.37197
[94]	validation_0-merror:0.17739	validation_1-merror:0.37173
[95]	validation_0-merror:0.17598	validation_1-merror:0.37146
[96]	validation_0-merror:0.17444	validation_1-merror:0.37105
[97]	validation_0-merror:0.17289	validation_1-merror:0.37075
[98]	validation_0-merror:0.17155	validation_1-merror:0.37061
[99]	validation_0-merror

[64]	validation_0-merror:0.21128	validation_1-merror:0.37726
[65]	validation_0-merror:0.20973	validation_1-merror:0.37657
[66]	validation_0-merror:0.20750	validation_1-merror:0.37648
[67]	validation_0-merror:0.20569	validation_1-merror:0.37546
[68]	validation_0-merror:0.20410	validation_1-merror:0.37508
[69]	validation_0-merror:0.20228	validation_1-merror:0.37434
[70]	validation_0-merror:0.20076	validation_1-merror:0.37418
[71]	validation_0-merror:0.19916	validation_1-merror:0.37385
[72]	validation_0-merror:0.19723	validation_1-merror:0.37379
[73]	validation_0-merror:0.19539	validation_1-merror:0.37328
[74]	validation_0-merror:0.19417	validation_1-merror:0.37314
[75]	validation_0-merror:0.19274	validation_1-merror:0.37330
[76]	validation_0-merror:0.19106	validation_1-merror:0.37340
[77]	validation_0-merror:0.18900	validation_1-merror:0.37322
[78]	validation_0-merror:0.18712	validation_1-merror:0.37259
[79]	validation_0-merror:0.18547	validation_1-merror:0.37271
[80]	validation_0-merror

[197]	validation_0-merror:0.06422	validation_1-merror:0.35568
[198]	validation_0-merror:0.06362	validation_1-merror:0.35532
[199]	validation_0-merror:0.06334	validation_1-merror:0.35517
[200]	validation_0-merror:0.06279	validation_1-merror:0.35544
[201]	validation_0-merror:0.06205	validation_1-merror:0.35581
[202]	validation_0-merror:0.06150	validation_1-merror:0.35550
[203]	validation_0-merror:0.06100	validation_1-merror:0.35515
[204]	validation_0-merror:0.06050	validation_1-merror:0.35509
[205]	validation_0-merror:0.05996	validation_1-merror:0.35440
[206]	validation_0-merror:0.05930	validation_1-merror:0.35485
[207]	validation_0-merror:0.05876	validation_1-merror:0.35485
[208]	validation_0-merror:0.05842	validation_1-merror:0.35479
[209]	validation_0-merror:0.05798	validation_1-merror:0.35489
[210]	validation_0-merror:0.05759	validation_1-merror:0.35511
[211]	validation_0-merror:0.05672	validation_1-merror:0.35509
[212]	validation_0-merror:0.05627	validation_1-merror:0.35524
[213]	va

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     eval_metric='merror', gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_con...
                                     num_class=19, num_parallel_tree=None,
                                     objective='multi:softmax',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=N

In [18]:
xgb_clf = XGBClassifier(objective="multi:softmax", num_class=19, nthread=-1, eval_metric= 'merror')
param_grid_dict = {
    "learning_rate": [0.05, 0.1, 0.2, 0.3, 0.4],
}
grid = GridSearchCV(xgb_clf, param_grid=param_grid_dict, cv=5)
grid.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=10)

[0]	validation_0-merror:0.50853	validation_1-merror:0.54940
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.49357	validation_1-merror:0.53307
[2]	validation_0-merror:0.48579	validation_1-merror:0.52903
[3]	validation_0-merror:0.47930	validation_1-merror:0.52374
[4]	validation_0-merror:0.47501	validation_1-merror:0.52111
[5]	validation_0-merror:0.47193	validation_1-merror:0.51666
[6]	validation_0-merror:0.46957	validation_1-merror:0.51386
[7]	validation_0-merror:0.46739	validation_1-merror:0.51249
[8]	validation_0-merror:0.46210	validation_1-merror:0.50825
[9]	validation_0-merror:0.45909	validation_1-merror:0.50584
[10]	validation_0-merror:0.45664	validation_1-merror:0.50453
[11]	validation_0-merror:0.45467	validation_1-merror:0.50141
[12]	validation_0-merror:0.45220	validation_1-merror:0.49996
[13]	validation_0-merror:0.44995	validation_1-merror:0

[30]	validation_0-merror:0.42058	validation_1-merror:0.47601
[31]	validation_0-merror:0.41886	validation_1-merror:0.47446
[32]	validation_0-merror:0.41648	validation_1-merror:0.47302
[33]	validation_0-merror:0.41555	validation_1-merror:0.47159
[34]	validation_0-merror:0.41332	validation_1-merror:0.46997
[35]	validation_0-merror:0.41263	validation_1-merror:0.46885
[36]	validation_0-merror:0.41140	validation_1-merror:0.46838
[37]	validation_0-merror:0.41049	validation_1-merror:0.46734
[38]	validation_0-merror:0.40976	validation_1-merror:0.46748
[39]	validation_0-merror:0.40863	validation_1-merror:0.46599
[40]	validation_0-merror:0.40833	validation_1-merror:0.46581
[41]	validation_0-merror:0.40710	validation_1-merror:0.46491
[42]	validation_0-merror:0.40612	validation_1-merror:0.46410
[43]	validation_0-merror:0.40507	validation_1-merror:0.46350
[44]	validation_0-merror:0.40373	validation_1-merror:0.46256
[45]	validation_0-merror:0.40282	validation_1-merror:0.46187
[46]	validation_0-merror

[62]	validation_0-merror:0.38483	validation_1-merror:0.44903
[63]	validation_0-merror:0.38427	validation_1-merror:0.44793
[64]	validation_0-merror:0.38333	validation_1-merror:0.44760
[65]	validation_0-merror:0.38271	validation_1-merror:0.44676
[66]	validation_0-merror:0.38179	validation_1-merror:0.44666
[67]	validation_0-merror:0.38081	validation_1-merror:0.44597
[68]	validation_0-merror:0.37990	validation_1-merror:0.44495
[69]	validation_0-merror:0.37890	validation_1-merror:0.44403
[70]	validation_0-merror:0.37777	validation_1-merror:0.44303
[71]	validation_0-merror:0.37656	validation_1-merror:0.44254
[72]	validation_0-merror:0.37575	validation_1-merror:0.44254
[73]	validation_0-merror:0.37503	validation_1-merror:0.44182
[74]	validation_0-merror:0.37367	validation_1-merror:0.44162
[75]	validation_0-merror:0.37303	validation_1-merror:0.44074
[76]	validation_0-merror:0.37215	validation_1-merror:0.43991
[77]	validation_0-merror:0.37132	validation_1-merror:0.43934
[78]	validation_0-merror

[94]	validation_0-merror:0.35940	validation_1-merror:0.43221
[95]	validation_0-merror:0.35871	validation_1-merror:0.43184
[96]	validation_0-merror:0.35775	validation_1-merror:0.43129
[97]	validation_0-merror:0.35697	validation_1-merror:0.43080
[98]	validation_0-merror:0.35589	validation_1-merror:0.43003
[99]	validation_0-merror:0.35530	validation_1-merror:0.42992
[0]	validation_0-merror:0.51674	validation_1-merror:0.55501
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.49454	validation_1-merror:0.53882
[2]	validation_0-merror:0.48858	validation_1-merror:0.52976
[3]	validation_0-merror:0.48012	validation_1-merror:0.52156
[4]	validation_0-merror:0.47687	validation_1-merror:0.51649
[5]	validation_0-merror:0.47640	validation_1-merror:0.51694
[6]	validation_0-merror:0.47231	validation_1-merror:0.51557
[7]	validation_0-merror:0.46909	validation_1-merror

[24]	validation_0-merror:0.39653	validation_1-merror:0.45999
[25]	validation_0-merror:0.39409	validation_1-merror:0.45807
[26]	validation_0-merror:0.39214	validation_1-merror:0.45658
[27]	validation_0-merror:0.39054	validation_1-merror:0.45522
[28]	validation_0-merror:0.38873	validation_1-merror:0.45424
[29]	validation_0-merror:0.38714	validation_1-merror:0.45244
[30]	validation_0-merror:0.38522	validation_1-merror:0.45119
[31]	validation_0-merror:0.38322	validation_1-merror:0.44899
[32]	validation_0-merror:0.38174	validation_1-merror:0.44793
[33]	validation_0-merror:0.37950	validation_1-merror:0.44685
[34]	validation_0-merror:0.37805	validation_1-merror:0.44521
[35]	validation_0-merror:0.37627	validation_1-merror:0.44356
[36]	validation_0-merror:0.37464	validation_1-merror:0.44203
[37]	validation_0-merror:0.37303	validation_1-merror:0.44097
[38]	validation_0-merror:0.37120	validation_1-merror:0.44029
[39]	validation_0-merror:0.36912	validation_1-merror:0.43909
[40]	validation_0-merror

...

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.47207	validation_1-merror:0.52233
[2]	validation_0-merror:0.45197	validation_1-merror:0.50663
[3]	validation_0-merror:0.43988	validation_1-merror:0.49986
[4]	validation_0-merror:0.42683	validation_1-merror:0.48845
[5]	validation_0-merror:0.41523	validation_1-merror:0.48071
[6]	validation_0-merror:0.40742	validation_1-merror:0.47318
[7]	validation_0-merror:0.39925	validation_1-merror:0.46687
[8]	validation_0-merror:0.38907	validation_1-merror:0.45803
[9]	validation_0-merror:0.38172	validation_1-merror:0.45256
[10]	validation_0-merror:0.37452	validation_1-merror:0.44973
[11]	validation_0-merror:0.36576	validation_1-merror:0.44354
[12]	validation_0-merror:0.35760	validation_1-merror:0.43793
[13]	validation_0-merror:0.35092	validation_1-merror:0.43388
[14]	validation_0-merror:0.34555	validation_1-merror:0.42943
[15]	validation_0-merror:0.34088	validation_1-merror:0.42649
[16]	validation_0-merror:0.

[32]	validation_0-merror:0.26981	validation_1-merror:0.39667
[33]	validation_0-merror:0.26692	validation_1-merror:0.39549
[34]	validation_0-merror:0.26298	validation_1-merror:0.39457
[35]	validation_0-merror:0.26018	validation_1-merror:0.39481
[36]	validation_0-merror:0.25741	validation_1-merror:0.39377
[37]	validation_0-merror:0.25425	validation_1-merror:0.39204
[38]	validation_0-merror:0.25044	validation_1-merror:0.39143
[39]	validation_0-merror:0.24766	validation_1-merror:0.38969
[40]	validation_0-merror:0.24491	validation_1-merror:0.38881
[41]	validation_0-merror:0.24269	validation_1-merror:0.38826
[42]	validation_0-merror:0.23932	validation_1-merror:0.38736
[43]	validation_0-merror:0.23674	validation_1-merror:0.38559
[44]	validation_0-merror:0.23376	validation_1-merror:0.38453
[45]	validation_0-merror:0.23128	validation_1-merror:0.38398
[46]	validation_0-merror:0.22904	validation_1-merror:0.38277
[47]	validation_0-merror:0.22549	validation_1-merror:0.38226
[48]	validation_0-merror

[64]	validation_0-merror:0.18708	validation_1-merror:0.37812
[65]	validation_0-merror:0.18496	validation_1-merror:0.37730
[66]	validation_0-merror:0.18304	validation_1-merror:0.37700
[67]	validation_0-merror:0.18079	validation_1-merror:0.37657
[68]	validation_0-merror:0.17908	validation_1-merror:0.37610
[69]	validation_0-merror:0.17677	validation_1-merror:0.37618
[70]	validation_0-merror:0.17490	validation_1-merror:0.37589
[71]	validation_0-merror:0.17360	validation_1-merror:0.37575
[72]	validation_0-merror:0.17242	validation_1-merror:0.37548
[73]	validation_0-merror:0.17090	validation_1-merror:0.37534
[74]	validation_0-merror:0.16895	validation_1-merror:0.37502
[75]	validation_0-merror:0.16771	validation_1-merror:0.37495
[76]	validation_0-merror:0.16585	validation_1-merror:0.37463
[77]	validation_0-merror:0.16447	validation_1-merror:0.37500
[78]	validation_0-merror:0.16311	validation_1-merror:0.37489
[79]	validation_0-merror:0.16179	validation_1-merror:0.37436
[80]	validation_0-merror

[96]	validation_0-merror:0.13695	validation_1-merror:0.37030
[97]	validation_0-merror:0.13573	validation_1-merror:0.37038
[98]	validation_0-merror:0.13433	validation_1-merror:0.36991
[99]	validation_0-merror:0.13278	validation_1-merror:0.36940
[0]	validation_0-merror:0.50699	validation_1-merror:0.54791
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.47120	validation_1-merror:0.51994
[2]	validation_0-merror:0.45176	validation_1-merror:0.50492
[3]	validation_0-merror:0.43670	validation_1-merror:0.49486
[4]	validation_0-merror:0.42649	validation_1-merror:0.48743
[5]	validation_0-merror:0.41465	validation_1-merror:0.47777
[6]	validation_0-merror:0.40409	validation_1-merror:0.46863
[7]	validation_0-merror:0.39581	validation_1-merror:0.46375
[8]	validation_0-merror:0.38724	validation_1-merror:0.45830
[9]	validation_0-merror:0.38017	validation_1-merror:0

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     eval_metric='merror', gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_con...
                                     num_class=19, num_parallel_tree=None,
                                     objective='multi:softmax',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=N

In [19]:
xgb_clf = XGBClassifier(objective="multi:softmax", num_class=19, nthread=-1, eval_metric= 'merror')
param_grid_dict = {
    "max_depth": [5, 6, 7],
}
grid = GridSearchCV(xgb_clf, param_grid=param_grid_dict, cv=5)
grid.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=10)

[0]	validation_0-merror:0.55432	validation_1-merror:0.58914
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.52221	validation_1-merror:0.55620
[2]	validation_0-merror:0.49622	validation_1-merror:0.53782
[3]	validation_0-merror:0.48825	validation_1-merror:0.53074
[4]	validation_0-merror:0.47796	validation_1-merror:0.52488
[5]	validation_0-merror:0.46873	validation_1-merror:0.51306
[6]	validation_0-merror:0.46161	validation_1-merror:0.50902
[7]	validation_0-merror:0.45327	validation_1-merror:0.49977
[8]	validation_0-merror:0.44772	validation_1-merror:0.49553
[9]	validation_0-merror:0.43836	validation_1-merror:0.48410
[10]	validation_0-merror:0.43432	validation_1-merror:0.48085
[11]	validation_0-merror:0.42746	validation_1-merror:0.47685
[12]	validation_0-merror:0.42204	validation_1-merror:0.47163
[13]	validation_0-merror:0.41568	validation_1-merror:0

[30]	validation_0-merror:0.34872	validation_1-merror:0.42470
[31]	validation_0-merror:0.34539	validation_1-merror:0.42388
[32]	validation_0-merror:0.34232	validation_1-merror:0.42264
[33]	validation_0-merror:0.33910	validation_1-merror:0.42158
[34]	validation_0-merror:0.33661	validation_1-merror:0.41955
[35]	validation_0-merror:0.33437	validation_1-merror:0.41886
[36]	validation_0-merror:0.33173	validation_1-merror:0.41725
[37]	validation_0-merror:0.32969	validation_1-merror:0.41637
[38]	validation_0-merror:0.32690	validation_1-merror:0.41537
[39]	validation_0-merror:0.32479	validation_1-merror:0.41441
[40]	validation_0-merror:0.32218	validation_1-merror:0.41292
[41]	validation_0-merror:0.32000	validation_1-merror:0.41064
[42]	validation_0-merror:0.31750	validation_1-merror:0.40974
[43]	validation_0-merror:0.31532	validation_1-merror:0.40869
[44]	validation_0-merror:0.31320	validation_1-merror:0.40741
[45]	validation_0-merror:0.31115	validation_1-merror:0.40721
[46]	validation_0-merror

[62]	validation_0-merror:0.28026	validation_1-merror:0.39161
[63]	validation_0-merror:0.27858	validation_1-merror:0.39141
[64]	validation_0-merror:0.27760	validation_1-merror:0.39090
[65]	validation_0-merror:0.27589	validation_1-merror:0.39088
[66]	validation_0-merror:0.27429	validation_1-merror:0.38994
[67]	validation_0-merror:0.27242	validation_1-merror:0.39000
[68]	validation_0-merror:0.27078	validation_1-merror:0.38987
[69]	validation_0-merror:0.26912	validation_1-merror:0.38892
[70]	validation_0-merror:0.26732	validation_1-merror:0.38869
[71]	validation_0-merror:0.26599	validation_1-merror:0.38883
[72]	validation_0-merror:0.26445	validation_1-merror:0.38798
[73]	validation_0-merror:0.26279	validation_1-merror:0.38824
[74]	validation_0-merror:0.26158	validation_1-merror:0.38724
[75]	validation_0-merror:0.26022	validation_1-merror:0.38692
[76]	validation_0-merror:0.25910	validation_1-merror:0.38649
[77]	validation_0-merror:0.25815	validation_1-merror:0.38606
[78]	validation_0-merror

[94]	validation_0-merror:0.23409	validation_1-merror:0.38014
[95]	validation_0-merror:0.23312	validation_1-merror:0.38040
[96]	validation_0-merror:0.23220	validation_1-merror:0.37959
[97]	validation_0-merror:0.23096	validation_1-merror:0.37894
[98]	validation_0-merror:0.22970	validation_1-merror:0.37840
[99]	validation_0-merror:0.22848	validation_1-merror:0.37845
[0]	validation_0-merror:0.56229	validation_1-merror:0.58786
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.52328	validation_1-merror:0.56030
[2]	validation_0-merror:0.50410	validation_1-merror:0.54597
[3]	validation_0-merror:0.49070	validation_1-merror:0.53388
[4]	validation_0-merror:0.48027	validation_1-merror:0.52270
[5]	validation_0-merror:0.47268	validation_1-merror:0.51380
[6]	validation_0-merror:0.46122	validation_1-merror:0.50088
[7]	validation_0-merror:0.45442	validation_1-merror

[24]	validation_0-merror:0.32492	validation_1-merror:0.41782
[25]	validation_0-merror:0.32199	validation_1-merror:0.41529
[26]	validation_0-merror:0.31787	validation_1-merror:0.41364
[27]	validation_0-merror:0.31463	validation_1-merror:0.41272
[28]	validation_0-merror:0.31104	validation_1-merror:0.41098
[29]	validation_0-merror:0.30786	validation_1-merror:0.41057
[30]	validation_0-merror:0.30476	validation_1-merror:0.40888
[31]	validation_0-merror:0.30247	validation_1-merror:0.40757
[32]	validation_0-merror:0.29956	validation_1-merror:0.40670
[33]	validation_0-merror:0.29661	validation_1-merror:0.40618
[34]	validation_0-merror:0.29310	validation_1-merror:0.40555
[35]	validation_0-merror:0.28996	validation_1-merror:0.40374
[36]	validation_0-merror:0.28708	validation_1-merror:0.40210
[37]	validation_0-merror:0.28485	validation_1-merror:0.40127
[38]	validation_0-merror:0.28116	validation_1-merror:0.40010
[39]	validation_0-merror:0.27913	validation_1-merror:0.39982
[40]	validation_0-merror

...

[82]	validation_0-merror:0.13609	validation_1-merror:0.36767
[83]	validation_0-merror:0.13480	validation_1-merror:0.36750
[84]	validation_0-merror:0.13256	validation_1-merror:0.36706
[85]	validation_0-merror:0.13098	validation_1-merror:0.36714
[86]	validation_0-merror:0.12929	validation_1-merror:0.36728
[87]	validation_0-merror:0.12804	validation_1-merror:0.36699
[88]	validation_0-merror:0.12715	validation_1-merror:0.36660
[89]	validation_0-merror:0.12584	validation_1-merror:0.36673
[90]	validation_0-merror:0.12420	validation_1-merror:0.36679
[91]	validation_0-merror:0.12259	validation_1-merror:0.36626
[92]	validation_0-merror:0.12128	validation_1-merror:0.36626
[93]	validation_0-merror:0.12050	validation_1-merror:0.36579
[94]	validation_0-merror:0.11907	validation_1-merror:0.36628
[95]	validation_0-merror:0.11789	validation_1-merror:0.36569
[96]	validation_0-merror:0.11662	validation_1-merror:0.36508
[97]	validation_0-merror:0.11553	validation_1-merror:0.36430
[98]	validation_0-merror

[12]	validation_0-merror:0.33506	validation_1-merror:0.42909
[13]	validation_0-merror:0.32976	validation_1-merror:0.42609
[14]	validation_0-merror:0.32380	validation_1-merror:0.42351
[15]	validation_0-merror:0.31839	validation_1-merror:0.42117
[16]	validation_0-merror:0.31455	validation_1-merror:0.41929
[17]	validation_0-merror:0.31017	validation_1-merror:0.41682
[18]	validation_0-merror:0.30572	validation_1-merror:0.41449
[19]	validation_0-merror:0.30148	validation_1-merror:0.41174
[20]	validation_0-merror:0.29677	validation_1-merror:0.40982
[21]	validation_0-merror:0.29279	validation_1-merror:0.40769
[22]	validation_0-merror:0.28820	validation_1-merror:0.40633
[23]	validation_0-merror:0.28418	validation_1-merror:0.40557
[24]	validation_0-merror:0.27951	validation_1-merror:0.40421
[25]	validation_0-merror:0.27458	validation_1-merror:0.40300
[26]	validation_0-merror:0.27150	validation_1-merror:0.40127
[27]	validation_0-merror:0.26756	validation_1-merror:0.39929
[28]	validation_0-merror

[44]	validation_0-merror:0.21310	validation_1-merror:0.38094
[45]	validation_0-merror:0.20959	validation_1-merror:0.37989
[46]	validation_0-merror:0.20675	validation_1-merror:0.37955
[47]	validation_0-merror:0.20390	validation_1-merror:0.37973
[48]	validation_0-merror:0.20089	validation_1-merror:0.37900
[49]	validation_0-merror:0.19872	validation_1-merror:0.37848
[50]	validation_0-merror:0.19626	validation_1-merror:0.37755
[51]	validation_0-merror:0.19404	validation_1-merror:0.37769
[52]	validation_0-merror:0.19223	validation_1-merror:0.37650
[53]	validation_0-merror:0.19027	validation_1-merror:0.37636
[54]	validation_0-merror:0.18824	validation_1-merror:0.37612
[55]	validation_0-merror:0.18564	validation_1-merror:0.37587
[56]	validation_0-merror:0.18343	validation_1-merror:0.37581
[57]	validation_0-merror:0.18026	validation_1-merror:0.37579
[58]	validation_0-merror:0.17825	validation_1-merror:0.37506
[59]	validation_0-merror:0.17658	validation_1-merror:0.37448
[60]	validation_0-merror

[76]	validation_0-merror:0.12986	validation_1-merror:0.36842
[77]	validation_0-merror:0.12834	validation_1-merror:0.36783
[78]	validation_0-merror:0.12684	validation_1-merror:0.36689
[79]	validation_0-merror:0.12503	validation_1-merror:0.36687
[80]	validation_0-merror:0.12317	validation_1-merror:0.36695
[81]	validation_0-merror:0.12095	validation_1-merror:0.36660
[82]	validation_0-merror:0.11900	validation_1-merror:0.36608
[83]	validation_0-merror:0.11763	validation_1-merror:0.36605
[84]	validation_0-merror:0.11624	validation_1-merror:0.36589
[85]	validation_0-merror:0.11443	validation_1-merror:0.36595
[86]	validation_0-merror:0.11301	validation_1-merror:0.36583
[87]	validation_0-merror:0.11182	validation_1-merror:0.36562
[88]	validation_0-merror:0.11035	validation_1-merror:0.36505
[89]	validation_0-merror:0.10869	validation_1-merror:0.36430
[90]	validation_0-merror:0.10730	validation_1-merror:0.36405
[91]	validation_0-merror:0.10508	validation_1-merror:0.36401
[92]	validation_0-merror

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     eval_metric='merror', gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_con...
                                     num_class=19, num_parallel_tree=None,
                                     objective='multi:softmax',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=N

In [20]:
xgb_clf = XGBClassifier(objective="multi:softmax", num_class=19, nthread=-1, eval_metric= 'merror')
param_grid_dict = {
    "subsample": [0.7, 0.8, 0.9, 1.0],
}
grid = GridSearchCV(xgb_clf, param_grid=param_grid_dict, cv=5)
grid.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=10)

[0]	validation_0-merror:0.50804	validation_1-merror:0.54583
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.47733	validation_1-merror:0.52149
[2]	validation_0-merror:0.46043	validation_1-merror:0.50947
[3]	validation_0-merror:0.44651	validation_1-merror:0.49873
[4]	validation_0-merror:0.43468	validation_1-merror:0.48906
[5]	validation_0-merror:0.42647	validation_1-merror:0.48136
[6]	validation_0-merror:0.41951	validation_1-merror:0.47650
[7]	validation_0-merror:0.41295	validation_1-merror:0.46967
[8]	validation_0-merror:0.40586	validation_1-merror:0.46495
[9]	validation_0-merror:0.39839	validation_1-merror:0.45952
[10]	validation_0-merror:0.38959	validation_1-merror:0.45399
[11]	validation_0-merror:0.38343	validation_1-merror:0.45027
[12]	validation_0-merror:0.37687	validation_1-merror:0.44603
[13]	validation_0-merror:0.37183	validation_1-merror:0

[30]	validation_0-merror:0.30233	validation_1-merror:0.40674
[31]	validation_0-merror:0.29876	validation_1-merror:0.40569
[32]	validation_0-merror:0.29507	validation_1-merror:0.40329
[33]	validation_0-merror:0.29224	validation_1-merror:0.40312
[34]	validation_0-merror:0.28980	validation_1-merror:0.40210
[35]	validation_0-merror:0.28670	validation_1-merror:0.40059
[36]	validation_0-merror:0.28383	validation_1-merror:0.39857
[37]	validation_0-merror:0.28205	validation_1-merror:0.39792
[38]	validation_0-merror:0.27923	validation_1-merror:0.39794
[39]	validation_0-merror:0.27631	validation_1-merror:0.39722
[40]	validation_0-merror:0.27343	validation_1-merror:0.39655
[41]	validation_0-merror:0.27122	validation_1-merror:0.39541
[42]	validation_0-merror:0.26873	validation_1-merror:0.39500
[43]	validation_0-merror:0.26644	validation_1-merror:0.39447
[44]	validation_0-merror:0.26399	validation_1-merror:0.39365
[45]	validation_0-merror:0.26175	validation_1-merror:0.39349
[46]	validation_0-merror

[62]	validation_0-merror:0.22101	validation_1-merror:0.38128
[63]	validation_0-merror:0.21906	validation_1-merror:0.38094
[64]	validation_0-merror:0.21757	validation_1-merror:0.38071
[65]	validation_0-merror:0.21551	validation_1-merror:0.37981
[66]	validation_0-merror:0.21326	validation_1-merror:0.37944
[67]	validation_0-merror:0.21103	validation_1-merror:0.37896
[68]	validation_0-merror:0.20969	validation_1-merror:0.37879
[69]	validation_0-merror:0.20768	validation_1-merror:0.37859
[70]	validation_0-merror:0.20570	validation_1-merror:0.37826
[71]	validation_0-merror:0.20409	validation_1-merror:0.37704
[72]	validation_0-merror:0.20239	validation_1-merror:0.37718
[73]	validation_0-merror:0.20092	validation_1-merror:0.37720
[74]	validation_0-merror:0.19959	validation_1-merror:0.37679
[75]	validation_0-merror:0.19780	validation_1-merror:0.37661
[76]	validation_0-merror:0.19598	validation_1-merror:0.37687
[77]	validation_0-merror:0.19439	validation_1-merror:0.37638
[78]	validation_0-merror

[94]	validation_0-merror:0.16720	validation_1-merror:0.37350
[95]	validation_0-merror:0.16590	validation_1-merror:0.37289
[96]	validation_0-merror:0.16446	validation_1-merror:0.37357
[97]	validation_0-merror:0.16289	validation_1-merror:0.37283
[98]	validation_0-merror:0.16171	validation_1-merror:0.37277
[99]	validation_0-merror:0.16001	validation_1-merror:0.37169
[0]	validation_0-merror:0.52189	validation_1-merror:0.56032
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.47981	validation_1-merror:0.52784
[2]	validation_0-merror:0.46580	validation_1-merror:0.51139
[3]	validation_0-merror:0.44847	validation_1-merror:0.49502
[4]	validation_0-merror:0.43996	validation_1-merror:0.48871
[5]	validation_0-merror:0.42810	validation_1-merror:0.47942
[6]	validation_0-merror:0.41945	validation_1-merror:0.47214
[7]	validation_0-merror:0.41282	validation_1-merror

[24]	validation_0-merror:0.32695	validation_1-merror:0.41941
[25]	validation_0-merror:0.32263	validation_1-merror:0.41758
[26]	validation_0-merror:0.31865	validation_1-merror:0.41523
[27]	validation_0-merror:0.31497	validation_1-merror:0.41302
[28]	validation_0-merror:0.31234	validation_1-merror:0.41125
[29]	validation_0-merror:0.30826	validation_1-merror:0.41127
[30]	validation_0-merror:0.30481	validation_1-merror:0.41037
[31]	validation_0-merror:0.30042	validation_1-merror:0.40894
[32]	validation_0-merror:0.29734	validation_1-merror:0.40633
[33]	validation_0-merror:0.29386	validation_1-merror:0.40525
[34]	validation_0-merror:0.29115	validation_1-merror:0.40363
[35]	validation_0-merror:0.28767	validation_1-merror:0.40249
[36]	validation_0-merror:0.28558	validation_1-merror:0.40192
[37]	validation_0-merror:0.28251	validation_1-merror:0.40029
[38]	validation_0-merror:0.28017	validation_1-merror:0.40012
[39]	validation_0-merror:0.27757	validation_1-merror:0.39792
[40]	validation_0-merror

...

[76]	validation_0-merror:0.20494	validation_1-merror:0.37867
[77]	validation_0-merror:0.20331	validation_1-merror:0.37863
[78]	validation_0-merror:0.20199	validation_1-merror:0.37836
[79]	validation_0-merror:0.20050	validation_1-merror:0.37802
[80]	validation_0-merror:0.19907	validation_1-merror:0.37736
[81]	validation_0-merror:0.19759	validation_1-merror:0.37728
[82]	validation_0-merror:0.19603	validation_1-merror:0.37712
[83]	validation_0-merror:0.19450	validation_1-merror:0.37636
[84]	validation_0-merror:0.19333	validation_1-merror:0.37638
[85]	validation_0-merror:0.19227	validation_1-merror:0.37634
[86]	validation_0-merror:0.19093	validation_1-merror:0.37636
[87]	validation_0-merror:0.18949	validation_1-merror:0.37616
[88]	validation_0-merror:0.18780	validation_1-merror:0.37597
[89]	validation_0-merror:0.18626	validation_1-merror:0.37504
[90]	validation_0-merror:0.18487	validation_1-merror:0.37487
[91]	validation_0-merror:0.18351	validation_1-merror:0.37465
[92]	validation_0-merror

[6]	validation_0-merror:0.41561	validation_1-merror:0.47099
[7]	validation_0-merror:0.40860	validation_1-merror:0.46581
[8]	validation_0-merror:0.40278	validation_1-merror:0.46105
[9]	validation_0-merror:0.39567	validation_1-merror:0.45736
[10]	validation_0-merror:0.38918	validation_1-merror:0.45303
[11]	validation_0-merror:0.38336	validation_1-merror:0.44736
[12]	validation_0-merror:0.37817	validation_1-merror:0.44305
[13]	validation_0-merror:0.37185	validation_1-merror:0.43915
[14]	validation_0-merror:0.36581	validation_1-merror:0.43599
[15]	validation_0-merror:0.35985	validation_1-merror:0.43235
[16]	validation_0-merror:0.35523	validation_1-merror:0.42992
[17]	validation_0-merror:0.35050	validation_1-merror:0.42700
[18]	validation_0-merror:0.34654	validation_1-merror:0.42511
[19]	validation_0-merror:0.34216	validation_1-merror:0.42229
[20]	validation_0-merror:0.33829	validation_1-merror:0.41962
[21]	validation_0-merror:0.33337	validation_1-merror:0.41725
[22]	validation_0-merror:0.3

[38]	validation_0-merror:0.27986	validation_1-merror:0.39686
[39]	validation_0-merror:0.27660	validation_1-merror:0.39602
[40]	validation_0-merror:0.27378	validation_1-merror:0.39582
[41]	validation_0-merror:0.27099	validation_1-merror:0.39422
[42]	validation_0-merror:0.26876	validation_1-merror:0.39333
[43]	validation_0-merror:0.26556	validation_1-merror:0.39198
[44]	validation_0-merror:0.26317	validation_1-merror:0.39147
[45]	validation_0-merror:0.26105	validation_1-merror:0.39116
[46]	validation_0-merror:0.25915	validation_1-merror:0.38984
[47]	validation_0-merror:0.25679	validation_1-merror:0.38804
[48]	validation_0-merror:0.25350	validation_1-merror:0.38692
[49]	validation_0-merror:0.25075	validation_1-merror:0.38588
[50]	validation_0-merror:0.24876	validation_1-merror:0.38500
[51]	validation_0-merror:0.24620	validation_1-merror:0.38453
[52]	validation_0-merror:0.24402	validation_1-merror:0.38361
[53]	validation_0-merror:0.24178	validation_1-merror:0.38298
[54]	validation_0-merror

[70]	validation_0-merror:0.20932	validation_1-merror:0.37693
[71]	validation_0-merror:0.20748	validation_1-merror:0.37591
[72]	validation_0-merror:0.20595	validation_1-merror:0.37550
[73]	validation_0-merror:0.20474	validation_1-merror:0.37550
[74]	validation_0-merror:0.20288	validation_1-merror:0.37534
[75]	validation_0-merror:0.20137	validation_1-merror:0.37548
[76]	validation_0-merror:0.19976	validation_1-merror:0.37461
[77]	validation_0-merror:0.19823	validation_1-merror:0.37461
[78]	validation_0-merror:0.19668	validation_1-merror:0.37444
[79]	validation_0-merror:0.19500	validation_1-merror:0.37414
[80]	validation_0-merror:0.19373	validation_1-merror:0.37410
[81]	validation_0-merror:0.19238	validation_1-merror:0.37387
[82]	validation_0-merror:0.19144	validation_1-merror:0.37369
[83]	validation_0-merror:0.19016	validation_1-merror:0.37334
[84]	validation_0-merror:0.18871	validation_1-merror:0.37310
[85]	validation_0-merror:0.18740	validation_1-merror:0.37281
[86]	validation_0-merror

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     eval_metric='merror', gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_con...
                                     num_class=19, num_parallel_tree=None,
                                     objective='multi:softmax',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=N

In [21]:
xgb_clf = XGBClassifier(objective="multi:softmax", num_class=19, nthread=-1, eval_metric= 'merror')
param_grid_dict = {
    "colsample_bytree": [0.7, 0.8, 0.9, 1.0],
}
grid = GridSearchCV(xgb_clf, param_grid=param_grid_dict, cv=5)
grid.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=10)

[0]	validation_0-merror:0.51671	validation_1-merror:0.55771
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.47831	validation_1-merror:0.52219
[2]	validation_0-merror:0.46222	validation_1-merror:0.51153
[3]	validation_0-merror:0.44891	validation_1-merror:0.50096
[4]	validation_0-merror:0.44015	validation_1-merror:0.49426
[5]	validation_0-merror:0.43002	validation_1-merror:0.48724
[6]	validation_0-merror:0.42243	validation_1-merror:0.48067
[7]	validation_0-merror:0.41453	validation_1-merror:0.47461
[8]	validation_0-merror:0.40786	validation_1-merror:0.46904
[9]	validation_0-merror:0.39980	validation_1-merror:0.46434
[10]	validation_0-merror:0.39351	validation_1-merror:0.45973
[11]	validation_0-merror:0.38749	validation_1-merror:0.45395
[12]	validation_0-merror:0.37944	validation_1-merror:0.45025
[13]	validation_0-merror:0.37352	validation_1-merror:0

[30]	validation_0-merror:0.30505	validation_1-merror:0.41043
[31]	validation_0-merror:0.30231	validation_1-merror:0.40861
[32]	validation_0-merror:0.29945	validation_1-merror:0.40678
[33]	validation_0-merror:0.29659	validation_1-merror:0.40582
[34]	validation_0-merror:0.29380	validation_1-merror:0.40455
[35]	validation_0-merror:0.29086	validation_1-merror:0.40435
[36]	validation_0-merror:0.28816	validation_1-merror:0.40370
[37]	validation_0-merror:0.28538	validation_1-merror:0.40251
[38]	validation_0-merror:0.28286	validation_1-merror:0.40198
[39]	validation_0-merror:0.28064	validation_1-merror:0.40076
[40]	validation_0-merror:0.27774	validation_1-merror:0.39920
[41]	validation_0-merror:0.27445	validation_1-merror:0.39833
[42]	validation_0-merror:0.27143	validation_1-merror:0.39675
[43]	validation_0-merror:0.26967	validation_1-merror:0.39604
[44]	validation_0-merror:0.26691	validation_1-merror:0.39457
[45]	validation_0-merror:0.26450	validation_1-merror:0.39449
[46]	validation_0-merror

[62]	validation_0-merror:0.22825	validation_1-merror:0.38371
[63]	validation_0-merror:0.22692	validation_1-merror:0.38334
[64]	validation_0-merror:0.22457	validation_1-merror:0.38294
[65]	validation_0-merror:0.22262	validation_1-merror:0.38140
[66]	validation_0-merror:0.22075	validation_1-merror:0.38132
[67]	validation_0-merror:0.21959	validation_1-merror:0.38094
[68]	validation_0-merror:0.21832	validation_1-merror:0.38067
[69]	validation_0-merror:0.21665	validation_1-merror:0.38063
[70]	validation_0-merror:0.21551	validation_1-merror:0.38026
[71]	validation_0-merror:0.21357	validation_1-merror:0.38018
[72]	validation_0-merror:0.21146	validation_1-merror:0.37934
[73]	validation_0-merror:0.21008	validation_1-merror:0.37896
[74]	validation_0-merror:0.20770	validation_1-merror:0.37814
[75]	validation_0-merror:0.20597	validation_1-merror:0.37722
[76]	validation_0-merror:0.20432	validation_1-merror:0.37748
[77]	validation_0-merror:0.20251	validation_1-merror:0.37744
[78]	validation_0-merror

[94]	validation_0-merror:0.17599	validation_1-merror:0.37200
[95]	validation_0-merror:0.17457	validation_1-merror:0.37148
[96]	validation_0-merror:0.17333	validation_1-merror:0.37128
[97]	validation_0-merror:0.17229	validation_1-merror:0.37118
[98]	validation_0-merror:0.17124	validation_1-merror:0.37048
[99]	validation_0-merror:0.17053	validation_1-merror:0.37006
[0]	validation_0-merror:0.51274	validation_1-merror:0.55185
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.48327	validation_1-merror:0.52678
[2]	validation_0-merror:0.46271	validation_1-merror:0.51411
[3]	validation_0-merror:0.44815	validation_1-merror:0.49943
[4]	validation_0-merror:0.43531	validation_1-merror:0.49196
[5]	validation_0-merror:0.42701	validation_1-merror:0.48396
[6]	validation_0-merror:0.41945	validation_1-merror:0.47702
[7]	validation_0-merror:0.41257	validation_1-merror

[24]	validation_0-merror:0.32531	validation_1-merror:0.41821
[25]	validation_0-merror:0.32236	validation_1-merror:0.41674
[26]	validation_0-merror:0.31933	validation_1-merror:0.41557
[27]	validation_0-merror:0.31634	validation_1-merror:0.41417
[28]	validation_0-merror:0.31168	validation_1-merror:0.41235
[29]	validation_0-merror:0.30899	validation_1-merror:0.41084
[30]	validation_0-merror:0.30508	validation_1-merror:0.41035
[31]	validation_0-merror:0.30182	validation_1-merror:0.40884
[32]	validation_0-merror:0.29944	validation_1-merror:0.40770
[33]	validation_0-merror:0.29531	validation_1-merror:0.40572
[34]	validation_0-merror:0.29238	validation_1-merror:0.40408
[35]	validation_0-merror:0.28992	validation_1-merror:0.40308
[36]	validation_0-merror:0.28658	validation_1-merror:0.40280
[37]	validation_0-merror:0.28331	validation_1-merror:0.40186
[38]	validation_0-merror:0.28043	validation_1-merror:0.40051
[39]	validation_0-merror:0.27750	validation_1-merror:0.39929
[40]	validation_0-merror

...

[76]	validation_0-merror:0.20494	validation_1-merror:0.37867
[77]	validation_0-merror:0.20331	validation_1-merror:0.37863
[78]	validation_0-merror:0.20199	validation_1-merror:0.37836
[79]	validation_0-merror:0.20050	validation_1-merror:0.37802
[80]	validation_0-merror:0.19907	validation_1-merror:0.37736
[81]	validation_0-merror:0.19759	validation_1-merror:0.37728
[82]	validation_0-merror:0.19603	validation_1-merror:0.37712
[83]	validation_0-merror:0.19450	validation_1-merror:0.37636
[84]	validation_0-merror:0.19333	validation_1-merror:0.37638
[85]	validation_0-merror:0.19227	validation_1-merror:0.37634
[86]	validation_0-merror:0.19093	validation_1-merror:0.37636
[87]	validation_0-merror:0.18949	validation_1-merror:0.37616
[88]	validation_0-merror:0.18780	validation_1-merror:0.37597
[89]	validation_0-merror:0.18626	validation_1-merror:0.37504
[90]	validation_0-merror:0.18487	validation_1-merror:0.37487
[91]	validation_0-merror:0.18351	validation_1-merror:0.37465
[92]	validation_0-merror

[6]	validation_0-merror:0.41561	validation_1-merror:0.47099
[7]	validation_0-merror:0.40860	validation_1-merror:0.46581
[8]	validation_0-merror:0.40278	validation_1-merror:0.46105
[9]	validation_0-merror:0.39567	validation_1-merror:0.45736
[10]	validation_0-merror:0.38918	validation_1-merror:0.45303
[11]	validation_0-merror:0.38336	validation_1-merror:0.44736
[12]	validation_0-merror:0.37817	validation_1-merror:0.44305
[13]	validation_0-merror:0.37185	validation_1-merror:0.43915
[14]	validation_0-merror:0.36581	validation_1-merror:0.43599
[15]	validation_0-merror:0.35985	validation_1-merror:0.43235
[16]	validation_0-merror:0.35523	validation_1-merror:0.42992
[17]	validation_0-merror:0.35050	validation_1-merror:0.42700
[18]	validation_0-merror:0.34654	validation_1-merror:0.42511
[19]	validation_0-merror:0.34216	validation_1-merror:0.42229
[20]	validation_0-merror:0.33829	validation_1-merror:0.41962
[21]	validation_0-merror:0.33337	validation_1-merror:0.41725
[22]	validation_0-merror:0.3

[38]	validation_0-merror:0.27986	validation_1-merror:0.39686
[39]	validation_0-merror:0.27660	validation_1-merror:0.39602
[40]	validation_0-merror:0.27378	validation_1-merror:0.39582
[41]	validation_0-merror:0.27099	validation_1-merror:0.39422
[42]	validation_0-merror:0.26876	validation_1-merror:0.39333
[43]	validation_0-merror:0.26556	validation_1-merror:0.39198
[44]	validation_0-merror:0.26317	validation_1-merror:0.39147
[45]	validation_0-merror:0.26105	validation_1-merror:0.39116
[46]	validation_0-merror:0.25915	validation_1-merror:0.38984
[47]	validation_0-merror:0.25679	validation_1-merror:0.38804
[48]	validation_0-merror:0.25350	validation_1-merror:0.38692
[49]	validation_0-merror:0.25075	validation_1-merror:0.38588
[50]	validation_0-merror:0.24876	validation_1-merror:0.38500
[51]	validation_0-merror:0.24620	validation_1-merror:0.38453
[52]	validation_0-merror:0.24402	validation_1-merror:0.38361
[53]	validation_0-merror:0.24178	validation_1-merror:0.38298
[54]	validation_0-merror

[70]	validation_0-merror:0.20932	validation_1-merror:0.37693
[71]	validation_0-merror:0.20748	validation_1-merror:0.37591
[72]	validation_0-merror:0.20595	validation_1-merror:0.37550
[73]	validation_0-merror:0.20474	validation_1-merror:0.37550
[74]	validation_0-merror:0.20288	validation_1-merror:0.37534
[75]	validation_0-merror:0.20137	validation_1-merror:0.37548
[76]	validation_0-merror:0.19976	validation_1-merror:0.37461
[77]	validation_0-merror:0.19823	validation_1-merror:0.37461
[78]	validation_0-merror:0.19668	validation_1-merror:0.37444
[79]	validation_0-merror:0.19500	validation_1-merror:0.37414
[80]	validation_0-merror:0.19373	validation_1-merror:0.37410
[81]	validation_0-merror:0.19238	validation_1-merror:0.37387
[82]	validation_0-merror:0.19144	validation_1-merror:0.37369
[83]	validation_0-merror:0.19016	validation_1-merror:0.37334
[84]	validation_0-merror:0.18871	validation_1-merror:0.37310
[85]	validation_0-merror:0.18740	validation_1-merror:0.37281
[86]	validation_0-merror

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     eval_metric='merror', gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_con...
                                     num_class=19, num_parallel_tree=None,
                                     objective='multi:softmax',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=N

In [22]:
xgb_clf = XGBClassifier(objective="multi:softmax", num_class=19, nthread=-1, eval_metric= 'merror')
param_grid_dict = {
    'min_child_weight': [0.5, 1, 1.5],
}
grid = GridSearchCV(xgb_clf, param_grid=param_grid_dict, cv=5)
grid.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=10)

[0]	validation_0-merror:0.50841	validation_1-merror:0.54960
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.47259	validation_1-merror:0.52025
[2]	validation_0-merror:0.45612	validation_1-merror:0.51172
[3]	validation_0-merror:0.44270	validation_1-merror:0.49873
[4]	validation_0-merror:0.43338	validation_1-merror:0.49157
[5]	validation_0-merror:0.42562	validation_1-merror:0.48573
[6]	validation_0-merror:0.41819	validation_1-merror:0.47871
[7]	validation_0-merror:0.41091	validation_1-merror:0.47246
[8]	validation_0-merror:0.40471	validation_1-merror:0.46789
[9]	validation_0-merror:0.39832	validation_1-merror:0.46346
[10]	validation_0-merror:0.39221	validation_1-merror:0.45889
[11]	validation_0-merror:0.38489	validation_1-merror:0.45217
[12]	validation_0-merror:0.37824	validation_1-merror:0.44781
[13]	validation_0-merror:0.37332	validation_1-merror:0

[30]	validation_0-merror:0.30215	validation_1-merror:0.40676
[31]	validation_0-merror:0.29932	validation_1-merror:0.40510
[32]	validation_0-merror:0.29590	validation_1-merror:0.40451
[33]	validation_0-merror:0.29310	validation_1-merror:0.40351
[34]	validation_0-merror:0.29062	validation_1-merror:0.40176
[35]	validation_0-merror:0.28657	validation_1-merror:0.40031
[36]	validation_0-merror:0.28334	validation_1-merror:0.39888
[37]	validation_0-merror:0.28119	validation_1-merror:0.39735
[38]	validation_0-merror:0.27892	validation_1-merror:0.39620
[39]	validation_0-merror:0.27601	validation_1-merror:0.39477
[40]	validation_0-merror:0.27332	validation_1-merror:0.39459
[41]	validation_0-merror:0.27103	validation_1-merror:0.39400
[42]	validation_0-merror:0.26801	validation_1-merror:0.39394
[43]	validation_0-merror:0.26603	validation_1-merror:0.39282
[44]	validation_0-merror:0.26359	validation_1-merror:0.39200
[45]	validation_0-merror:0.26027	validation_1-merror:0.39106
[46]	validation_0-merror

[62]	validation_0-merror:0.22278	validation_1-merror:0.37708
[63]	validation_0-merror:0.22072	validation_1-merror:0.37708
[64]	validation_0-merror:0.21890	validation_1-merror:0.37636
[65]	validation_0-merror:0.21735	validation_1-merror:0.37653
[66]	validation_0-merror:0.21573	validation_1-merror:0.37583
[67]	validation_0-merror:0.21395	validation_1-merror:0.37548
[68]	validation_0-merror:0.21162	validation_1-merror:0.37512
[69]	validation_0-merror:0.21030	validation_1-merror:0.37495
[70]	validation_0-merror:0.20869	validation_1-merror:0.37479
[71]	validation_0-merror:0.20686	validation_1-merror:0.37481
[72]	validation_0-merror:0.20570	validation_1-merror:0.37400
[73]	validation_0-merror:0.20386	validation_1-merror:0.37310
[74]	validation_0-merror:0.20254	validation_1-merror:0.37287
[75]	validation_0-merror:0.20107	validation_1-merror:0.37318
[76]	validation_0-merror:0.19913	validation_1-merror:0.37318
[77]	validation_0-merror:0.19748	validation_1-merror:0.37275
[78]	validation_0-merror

[94]	validation_0-merror:0.17271	validation_1-merror:0.37126
[95]	validation_0-merror:0.17191	validation_1-merror:0.37112
[96]	validation_0-merror:0.17044	validation_1-merror:0.37124
[97]	validation_0-merror:0.16944	validation_1-merror:0.37126
[98]	validation_0-merror:0.16841	validation_1-merror:0.37116
[99]	validation_0-merror:0.16737	validation_1-merror:0.37069
[0]	validation_0-merror:0.51662	validation_1-merror:0.55499
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.47816	validation_1-merror:0.52278
[2]	validation_0-merror:0.46183	validation_1-merror:0.51008
[3]	validation_0-merror:0.44783	validation_1-merror:0.50337
[4]	validation_0-merror:0.43431	validation_1-merror:0.49235
[5]	validation_0-merror:0.42631	validation_1-merror:0.48645
[6]	validation_0-merror:0.41879	validation_1-merror:0.47857
[7]	validation_0-merror:0.41046	validation_1-merror

[24]	validation_0-merror:0.32492	validation_1-merror:0.41782
[25]	validation_0-merror:0.32199	validation_1-merror:0.41529
[26]	validation_0-merror:0.31787	validation_1-merror:0.41364
[27]	validation_0-merror:0.31463	validation_1-merror:0.41272
[28]	validation_0-merror:0.31104	validation_1-merror:0.41098
[29]	validation_0-merror:0.30786	validation_1-merror:0.41057
[30]	validation_0-merror:0.30476	validation_1-merror:0.40888
[31]	validation_0-merror:0.30247	validation_1-merror:0.40757
[32]	validation_0-merror:0.29956	validation_1-merror:0.40670
[33]	validation_0-merror:0.29661	validation_1-merror:0.40618
[34]	validation_0-merror:0.29310	validation_1-merror:0.40555
[35]	validation_0-merror:0.28996	validation_1-merror:0.40374
[36]	validation_0-merror:0.28708	validation_1-merror:0.40210
[37]	validation_0-merror:0.28485	validation_1-merror:0.40127
[38]	validation_0-merror:0.28116	validation_1-merror:0.40010
[39]	validation_0-merror:0.27913	validation_1-merror:0.39982
[40]	validation_0-merror

...

[82]	validation_0-merror:0.19329	validation_1-merror:0.37614
[83]	validation_0-merror:0.19153	validation_1-merror:0.37538
[84]	validation_0-merror:0.19030	validation_1-merror:0.37497
[85]	validation_0-merror:0.18890	validation_1-merror:0.37485
[86]	validation_0-merror:0.18759	validation_1-merror:0.37497
[87]	validation_0-merror:0.18666	validation_1-merror:0.37504
[88]	validation_0-merror:0.18507	validation_1-merror:0.37463
[89]	validation_0-merror:0.18341	validation_1-merror:0.37448
[90]	validation_0-merror:0.18162	validation_1-merror:0.37444
[91]	validation_0-merror:0.18018	validation_1-merror:0.37483
[92]	validation_0-merror:0.17932	validation_1-merror:0.37444
[93]	validation_0-merror:0.17806	validation_1-merror:0.37361
[94]	validation_0-merror:0.17684	validation_1-merror:0.37342
[95]	validation_0-merror:0.17593	validation_1-merror:0.37306
[96]	validation_0-merror:0.17462	validation_1-merror:0.37336
[97]	validation_0-merror:0.17321	validation_1-merror:0.37355
[98]	validation_0-merror

[12]	validation_0-merror:0.37944	validation_1-merror:0.45062
[13]	validation_0-merror:0.37375	validation_1-merror:0.44658
[14]	validation_0-merror:0.36852	validation_1-merror:0.44280
[15]	validation_0-merror:0.36347	validation_1-merror:0.43888
[16]	validation_0-merror:0.35850	validation_1-merror:0.43566
[17]	validation_0-merror:0.35354	validation_1-merror:0.43340
[18]	validation_0-merror:0.34891	validation_1-merror:0.43105
[19]	validation_0-merror:0.34536	validation_1-merror:0.42921
[20]	validation_0-merror:0.34149	validation_1-merror:0.42700
[21]	validation_0-merror:0.33541	validation_1-merror:0.42417
[22]	validation_0-merror:0.33089	validation_1-merror:0.42111
[23]	validation_0-merror:0.32693	validation_1-merror:0.41857
[24]	validation_0-merror:0.32276	validation_1-merror:0.41774
[25]	validation_0-merror:0.31930	validation_1-merror:0.41498
[26]	validation_0-merror:0.31608	validation_1-merror:0.41333
[27]	validation_0-merror:0.31280	validation_1-merror:0.41190
[28]	validation_0-merror

[44]	validation_0-merror:0.26449	validation_1-merror:0.39263
[45]	validation_0-merror:0.26221	validation_1-merror:0.39149
[46]	validation_0-merror:0.25999	validation_1-merror:0.39037
[47]	validation_0-merror:0.25811	validation_1-merror:0.39002
[48]	validation_0-merror:0.25593	validation_1-merror:0.38904
[49]	validation_0-merror:0.25384	validation_1-merror:0.38884
[50]	validation_0-merror:0.25107	validation_1-merror:0.38867
[51]	validation_0-merror:0.24936	validation_1-merror:0.38792
[52]	validation_0-merror:0.24730	validation_1-merror:0.38687
[53]	validation_0-merror:0.24505	validation_1-merror:0.38622
[54]	validation_0-merror:0.24299	validation_1-merror:0.38549
[55]	validation_0-merror:0.24106	validation_1-merror:0.38510
[56]	validation_0-merror:0.23866	validation_1-merror:0.38416
[57]	validation_0-merror:0.23662	validation_1-merror:0.38324
[58]	validation_0-merror:0.23486	validation_1-merror:0.38326
[59]	validation_0-merror:0.23245	validation_1-merror:0.38257
[60]	validation_0-merror

[76]	validation_0-merror:0.19095	validation_1-merror:0.37344
[77]	validation_0-merror:0.18930	validation_1-merror:0.37393
[78]	validation_0-merror:0.18763	validation_1-merror:0.37340
[79]	validation_0-merror:0.18602	validation_1-merror:0.37293
[80]	validation_0-merror:0.18454	validation_1-merror:0.37302
[81]	validation_0-merror:0.18272	validation_1-merror:0.37224
[82]	validation_0-merror:0.18109	validation_1-merror:0.37142
[83]	validation_0-merror:0.17994	validation_1-merror:0.37193
[84]	validation_0-merror:0.17836	validation_1-merror:0.37189
[85]	validation_0-merror:0.17691	validation_1-merror:0.37154
[86]	validation_0-merror:0.17516	validation_1-merror:0.37122
[87]	validation_0-merror:0.17371	validation_1-merror:0.37124
[88]	validation_0-merror:0.17148	validation_1-merror:0.37077
[89]	validation_0-merror:0.17045	validation_1-merror:0.36995
[90]	validation_0-merror:0.16899	validation_1-merror:0.37030
[91]	validation_0-merror:0.16747	validation_1-merror:0.37036
[92]	validation_0-merror

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     eval_metric='merror', gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_con...
                                     num_class=19, num_parallel_tree=None,
                                     objective='multi:softmax',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=N

In [23]:
xgb_clf = XGBClassifier(objective="multi:softmax", num_class=19, nthread=-1, eval_metric= 'merror')
param_grid_dict = {
    'gamma': [0, 0.1, 0.2],
}
grid = GridSearchCV(xgb_clf, param_grid=param_grid_dict, cv=5)
grid.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=10)

[0]	validation_0-merror:0.50853	validation_1-merror:0.54940
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.47253	validation_1-merror:0.51992
[2]	validation_0-merror:0.45688	validation_1-merror:0.51174
[3]	validation_0-merror:0.44457	validation_1-merror:0.50029
[4]	validation_0-merror:0.43396	validation_1-merror:0.49043
[5]	validation_0-merror:0.42551	validation_1-merror:0.48475
[6]	validation_0-merror:0.41904	validation_1-merror:0.48006
[7]	validation_0-merror:0.41089	validation_1-merror:0.47301
[8]	validation_0-merror:0.40409	validation_1-merror:0.46618
[9]	validation_0-merror:0.39766	validation_1-merror:0.46214
[10]	validation_0-merror:0.39115	validation_1-merror:0.45736
[11]	validation_0-merror:0.38608	validation_1-merror:0.45283
[12]	validation_0-merror:0.38039	validation_1-merror:0.45030
[13]	validation_0-merror:0.37500	validation_1-merror:0

[30]	validation_0-merror:0.30329	validation_1-merror:0.40451
[31]	validation_0-merror:0.30048	validation_1-merror:0.40308
[32]	validation_0-merror:0.29826	validation_1-merror:0.40200
[33]	validation_0-merror:0.29511	validation_1-merror:0.40139
[34]	validation_0-merror:0.29208	validation_1-merror:0.40049
[35]	validation_0-merror:0.28876	validation_1-merror:0.39943
[36]	validation_0-merror:0.28527	validation_1-merror:0.39816
[37]	validation_0-merror:0.28219	validation_1-merror:0.39720
[38]	validation_0-merror:0.27913	validation_1-merror:0.39649
[39]	validation_0-merror:0.27686	validation_1-merror:0.39602
[40]	validation_0-merror:0.27399	validation_1-merror:0.39486
[41]	validation_0-merror:0.27139	validation_1-merror:0.39406
[42]	validation_0-merror:0.26838	validation_1-merror:0.39335
[43]	validation_0-merror:0.26608	validation_1-merror:0.39233
[44]	validation_0-merror:0.26406	validation_1-merror:0.39108
[45]	validation_0-merror:0.26128	validation_1-merror:0.38988
[46]	validation_0-merror

[62]	validation_0-merror:0.22345	validation_1-merror:0.37661
[63]	validation_0-merror:0.22150	validation_1-merror:0.37657
[64]	validation_0-merror:0.21972	validation_1-merror:0.37700
[65]	validation_0-merror:0.21743	validation_1-merror:0.37648
[66]	validation_0-merror:0.21556	validation_1-merror:0.37569
[67]	validation_0-merror:0.21435	validation_1-merror:0.37538
[68]	validation_0-merror:0.21281	validation_1-merror:0.37463
[69]	validation_0-merror:0.21190	validation_1-merror:0.37479
[70]	validation_0-merror:0.21037	validation_1-merror:0.37418
[71]	validation_0-merror:0.20908	validation_1-merror:0.37357
[72]	validation_0-merror:0.20710	validation_1-merror:0.37355
[73]	validation_0-merror:0.20550	validation_1-merror:0.37263
[74]	validation_0-merror:0.20389	validation_1-merror:0.37218
[75]	validation_0-merror:0.20218	validation_1-merror:0.37197
[76]	validation_0-merror:0.20089	validation_1-merror:0.37134
[77]	validation_0-merror:0.19957	validation_1-merror:0.37146
[78]	validation_0-merror

[94]	validation_0-merror:0.17484	validation_1-merror:0.37177
[95]	validation_0-merror:0.17375	validation_1-merror:0.37144
[96]	validation_0-merror:0.17232	validation_1-merror:0.37128
[97]	validation_0-merror:0.17105	validation_1-merror:0.37156
[98]	validation_0-merror:0.16936	validation_1-merror:0.37146
[99]	validation_0-merror:0.16772	validation_1-merror:0.37097
[0]	validation_0-merror:0.51674	validation_1-merror:0.55501
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.47832	validation_1-merror:0.52286
[2]	validation_0-merror:0.46200	validation_1-merror:0.51006
[3]	validation_0-merror:0.44838	validation_1-merror:0.50302
[4]	validation_0-merror:0.43559	validation_1-merror:0.49202
[5]	validation_0-merror:0.42797	validation_1-merror:0.48794
[6]	validation_0-merror:0.41958	validation_1-merror:0.48049
[7]	validation_0-merror:0.41206	validation_1-merror

[24]	validation_0-merror:0.32535	validation_1-merror:0.41841
[25]	validation_0-merror:0.32158	validation_1-merror:0.41592
[26]	validation_0-merror:0.31802	validation_1-merror:0.41496
[27]	validation_0-merror:0.31461	validation_1-merror:0.41253
[28]	validation_0-merror:0.31168	validation_1-merror:0.41131
[29]	validation_0-merror:0.30713	validation_1-merror:0.40992
[30]	validation_0-merror:0.30402	validation_1-merror:0.40815
[31]	validation_0-merror:0.30115	validation_1-merror:0.40812
[32]	validation_0-merror:0.29838	validation_1-merror:0.40625
[33]	validation_0-merror:0.29470	validation_1-merror:0.40396
[34]	validation_0-merror:0.29148	validation_1-merror:0.40269
[35]	validation_0-merror:0.28907	validation_1-merror:0.40200
[36]	validation_0-merror:0.28659	validation_1-merror:0.40078
[37]	validation_0-merror:0.28343	validation_1-merror:0.39984
[38]	validation_0-merror:0.28042	validation_1-merror:0.39816
[39]	validation_0-merror:0.27734	validation_1-merror:0.39702
[40]	validation_0-merror

...

[82]	validation_0-merror:0.19096	validation_1-merror:0.37469
[83]	validation_0-merror:0.18974	validation_1-merror:0.37485
[84]	validation_0-merror:0.18816	validation_1-merror:0.37444
[85]	validation_0-merror:0.18670	validation_1-merror:0.37416
[86]	validation_0-merror:0.18549	validation_1-merror:0.37450
[87]	validation_0-merror:0.18426	validation_1-merror:0.37426
[88]	validation_0-merror:0.18235	validation_1-merror:0.37357
[89]	validation_0-merror:0.18125	validation_1-merror:0.37350
[90]	validation_0-merror:0.17971	validation_1-merror:0.37312
[91]	validation_0-merror:0.17792	validation_1-merror:0.37234
[92]	validation_0-merror:0.17669	validation_1-merror:0.37236
[93]	validation_0-merror:0.17495	validation_1-merror:0.37224
[94]	validation_0-merror:0.17347	validation_1-merror:0.37193
[95]	validation_0-merror:0.17188	validation_1-merror:0.37126
[96]	validation_0-merror:0.17038	validation_1-merror:0.37032
[97]	validation_0-merror:0.16939	validation_1-merror:0.37024
[98]	validation_0-merror

[12]	validation_0-merror:0.37795	validation_1-merror:0.44850
[13]	validation_0-merror:0.37320	validation_1-merror:0.44497
[14]	validation_0-merror:0.36721	validation_1-merror:0.44236
[15]	validation_0-merror:0.36226	validation_1-merror:0.43995
[16]	validation_0-merror:0.35738	validation_1-merror:0.43725
[17]	validation_0-merror:0.35321	validation_1-merror:0.43491
[18]	validation_0-merror:0.34732	validation_1-merror:0.43203
[19]	validation_0-merror:0.34291	validation_1-merror:0.43033
[20]	validation_0-merror:0.33834	validation_1-merror:0.42666
[21]	validation_0-merror:0.33320	validation_1-merror:0.42401
[22]	validation_0-merror:0.33078	validation_1-merror:0.42317
[23]	validation_0-merror:0.32580	validation_1-merror:0.42006
[24]	validation_0-merror:0.32224	validation_1-merror:0.41817
[25]	validation_0-merror:0.31883	validation_1-merror:0.41543
[26]	validation_0-merror:0.31634	validation_1-merror:0.41376
[27]	validation_0-merror:0.31367	validation_1-merror:0.41331
[28]	validation_0-merror

[44]	validation_0-merror:0.26125	validation_1-merror:0.38998
[45]	validation_0-merror:0.25870	validation_1-merror:0.38969
[46]	validation_0-merror:0.25696	validation_1-merror:0.38879
[47]	validation_0-merror:0.25415	validation_1-merror:0.38828
[48]	validation_0-merror:0.25190	validation_1-merror:0.38739
[49]	validation_0-merror:0.24923	validation_1-merror:0.38683
[50]	validation_0-merror:0.24725	validation_1-merror:0.38590
[51]	validation_0-merror:0.24509	validation_1-merror:0.38547
[52]	validation_0-merror:0.24277	validation_1-merror:0.38422
[53]	validation_0-merror:0.24081	validation_1-merror:0.38387
[54]	validation_0-merror:0.23861	validation_1-merror:0.38238
[55]	validation_0-merror:0.23670	validation_1-merror:0.38271
[56]	validation_0-merror:0.23460	validation_1-merror:0.38189
[57]	validation_0-merror:0.23286	validation_1-merror:0.38142
[58]	validation_0-merror:0.23093	validation_1-merror:0.38112
[59]	validation_0-merror:0.22886	validation_1-merror:0.38051
[60]	validation_0-merror

[76]	validation_0-merror:0.19288	validation_1-merror:0.37457
[77]	validation_0-merror:0.19099	validation_1-merror:0.37416
[78]	validation_0-merror:0.18923	validation_1-merror:0.37412
[79]	validation_0-merror:0.18788	validation_1-merror:0.37406
[80]	validation_0-merror:0.18624	validation_1-merror:0.37330
[81]	validation_0-merror:0.18448	validation_1-merror:0.37357
[82]	validation_0-merror:0.18244	validation_1-merror:0.37308
[83]	validation_0-merror:0.18099	validation_1-merror:0.37228
[84]	validation_0-merror:0.17921	validation_1-merror:0.37179
[85]	validation_0-merror:0.17769	validation_1-merror:0.37197
[86]	validation_0-merror:0.17619	validation_1-merror:0.37236
[87]	validation_0-merror:0.17525	validation_1-merror:0.37212
[88]	validation_0-merror:0.17349	validation_1-merror:0.37167
[89]	validation_0-merror:0.17196	validation_1-merror:0.37159
[90]	validation_0-merror:0.17048	validation_1-merror:0.37108
[91]	validation_0-merror:0.16905	validation_1-merror:0.37077
[92]	validation_0-merror

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     eval_metric='merror', gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_con...
                                     num_class=19, num_parallel_tree=None,
                                     objective='multi:softmax',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=N

In [24]:
xgb_clf = XGBClassifier(objective="multi:softmax", num_class=19, nthread=-1, eval_metric= 'merror')
param_grid_dict = {
    'reg_lambda': [0.8, 1, 1.5, 2]
}
grid = GridSearchCV(xgb_clf, param_grid=param_grid_dict, cv=5)
grid.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], early_stopping_rounds=10)

[0]	validation_0-merror:0.50933	validation_1-merror:0.54774
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.47187	validation_1-merror:0.51874
[2]	validation_0-merror:0.45620	validation_1-merror:0.50890
[3]	validation_0-merror:0.44320	validation_1-merror:0.49841
[4]	validation_0-merror:0.43213	validation_1-merror:0.48706
[5]	validation_0-merror:0.42394	validation_1-merror:0.48159
[6]	validation_0-merror:0.41724	validation_1-merror:0.47597
[7]	validation_0-merror:0.40952	validation_1-merror:0.46838
[8]	validation_0-merror:0.40235	validation_1-merror:0.46418
[9]	validation_0-merror:0.39531	validation_1-merror:0.45897
[10]	validation_0-merror:0.38837	validation_1-merror:0.45573
[11]	validation_0-merror:0.38354	validation_1-merror:0.45085
[12]	validation_0-merror:0.37774	validation_1-merror:0.44727
[13]	validation_0-merror:0.37226	validation_1-merror:0

[30]	validation_0-merror:0.30233	validation_1-merror:0.40608
[31]	validation_0-merror:0.29890	validation_1-merror:0.40414
[32]	validation_0-merror:0.29651	validation_1-merror:0.40325
[33]	validation_0-merror:0.29288	validation_1-merror:0.40225
[34]	validation_0-merror:0.28946	validation_1-merror:0.40047
[35]	validation_0-merror:0.28621	validation_1-merror:0.39951
[36]	validation_0-merror:0.28333	validation_1-merror:0.39792
[37]	validation_0-merror:0.28057	validation_1-merror:0.39702
[38]	validation_0-merror:0.27721	validation_1-merror:0.39606
[39]	validation_0-merror:0.27478	validation_1-merror:0.39481
[40]	validation_0-merror:0.27260	validation_1-merror:0.39410
[41]	validation_0-merror:0.26981	validation_1-merror:0.39371
[42]	validation_0-merror:0.26735	validation_1-merror:0.39265
[43]	validation_0-merror:0.26522	validation_1-merror:0.39212
[44]	validation_0-merror:0.26261	validation_1-merror:0.39126
[45]	validation_0-merror:0.26098	validation_1-merror:0.39120
[46]	validation_0-merror

[62]	validation_0-merror:0.22510	validation_1-merror:0.37873
[63]	validation_0-merror:0.22359	validation_1-merror:0.37849
[64]	validation_0-merror:0.22168	validation_1-merror:0.37830
[65]	validation_0-merror:0.21981	validation_1-merror:0.37785
[66]	validation_0-merror:0.21793	validation_1-merror:0.37757
[67]	validation_0-merror:0.21658	validation_1-merror:0.37730
[68]	validation_0-merror:0.21538	validation_1-merror:0.37708
[69]	validation_0-merror:0.21379	validation_1-merror:0.37704
[70]	validation_0-merror:0.21145	validation_1-merror:0.37589
[71]	validation_0-merror:0.20972	validation_1-merror:0.37587
[72]	validation_0-merror:0.20783	validation_1-merror:0.37587
[73]	validation_0-merror:0.20595	validation_1-merror:0.37546
[74]	validation_0-merror:0.20476	validation_1-merror:0.37479
[75]	validation_0-merror:0.20327	validation_1-merror:0.37444
[76]	validation_0-merror:0.20177	validation_1-merror:0.37385
[77]	validation_0-merror:0.20030	validation_1-merror:0.37381
[78]	validation_0-merror

[94]	validation_0-merror:0.17309	validation_1-merror:0.37177
[95]	validation_0-merror:0.17182	validation_1-merror:0.37126
[96]	validation_0-merror:0.17042	validation_1-merror:0.37206
[97]	validation_0-merror:0.16925	validation_1-merror:0.37146
[98]	validation_0-merror:0.16789	validation_1-merror:0.37108
[99]	validation_0-merror:0.16662	validation_1-merror:0.37095
[0]	validation_0-merror:0.51673	validation_1-merror:0.55619
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.47687	validation_1-merror:0.52176
[2]	validation_0-merror:0.45985	validation_1-merror:0.50953
[3]	validation_0-merror:0.44591	validation_1-merror:0.49912
[4]	validation_0-merror:0.43496	validation_1-merror:0.49134
[5]	validation_0-merror:0.42612	validation_1-merror:0.48377
[6]	validation_0-merror:0.41789	validation_1-merror:0.47714
[7]	validation_0-merror:0.41269	validation_1-merror

[24]	validation_0-merror:0.32492	validation_1-merror:0.41782
[25]	validation_0-merror:0.32199	validation_1-merror:0.41529
[26]	validation_0-merror:0.31787	validation_1-merror:0.41364
[27]	validation_0-merror:0.31463	validation_1-merror:0.41272
[28]	validation_0-merror:0.31104	validation_1-merror:0.41098
[29]	validation_0-merror:0.30786	validation_1-merror:0.41057
[30]	validation_0-merror:0.30476	validation_1-merror:0.40888
[31]	validation_0-merror:0.30247	validation_1-merror:0.40757
[32]	validation_0-merror:0.29956	validation_1-merror:0.40670
[33]	validation_0-merror:0.29661	validation_1-merror:0.40618
[34]	validation_0-merror:0.29310	validation_1-merror:0.40555
[35]	validation_0-merror:0.28996	validation_1-merror:0.40374
[36]	validation_0-merror:0.28708	validation_1-merror:0.40210
[37]	validation_0-merror:0.28485	validation_1-merror:0.40127
[38]	validation_0-merror:0.28116	validation_1-merror:0.40010
[39]	validation_0-merror:0.27913	validation_1-merror:0.39982
[40]	validation_0-merror

...

[76]	validation_0-merror:0.20693	validation_1-merror:0.37838
[77]	validation_0-merror:0.20532	validation_1-merror:0.37810
[78]	validation_0-merror:0.20384	validation_1-merror:0.37775
[79]	validation_0-merror:0.20268	validation_1-merror:0.37757
[80]	validation_0-merror:0.20105	validation_1-merror:0.37671
[81]	validation_0-merror:0.19930	validation_1-merror:0.37663
[82]	validation_0-merror:0.19789	validation_1-merror:0.37634
[83]	validation_0-merror:0.19635	validation_1-merror:0.37597
[84]	validation_0-merror:0.19490	validation_1-merror:0.37597
[85]	validation_0-merror:0.19330	validation_1-merror:0.37599
[86]	validation_0-merror:0.19192	validation_1-merror:0.37540
[87]	validation_0-merror:0.19021	validation_1-merror:0.37536
[88]	validation_0-merror:0.18868	validation_1-merror:0.37483
[89]	validation_0-merror:0.18710	validation_1-merror:0.37473
[90]	validation_0-merror:0.18531	validation_1-merror:0.37379
[91]	validation_0-merror:0.18401	validation_1-merror:0.37416
[92]	validation_0-merror

[6]	validation_0-merror:0.41893	validation_1-merror:0.47477
[7]	validation_0-merror:0.41168	validation_1-merror:0.46863
[8]	validation_0-merror:0.40489	validation_1-merror:0.46346
[9]	validation_0-merror:0.39655	validation_1-merror:0.45685
[10]	validation_0-merror:0.39104	validation_1-merror:0.45260
[11]	validation_0-merror:0.38603	validation_1-merror:0.44958
[12]	validation_0-merror:0.38069	validation_1-merror:0.44707
[13]	validation_0-merror:0.37486	validation_1-merror:0.44513
[14]	validation_0-merror:0.36810	validation_1-merror:0.44074
[15]	validation_0-merror:0.36407	validation_1-merror:0.43780
[16]	validation_0-merror:0.35937	validation_1-merror:0.43529
[17]	validation_0-merror:0.35449	validation_1-merror:0.43325
[18]	validation_0-merror:0.35082	validation_1-merror:0.43054
[19]	validation_0-merror:0.34653	validation_1-merror:0.42790
[20]	validation_0-merror:0.34202	validation_1-merror:0.42521
[21]	validation_0-merror:0.33847	validation_1-merror:0.42347
[22]	validation_0-merror:0.3

[38]	validation_0-merror:0.28157	validation_1-merror:0.39747
[39]	validation_0-merror:0.27858	validation_1-merror:0.39635
[40]	validation_0-merror:0.27658	validation_1-merror:0.39582
[41]	validation_0-merror:0.27397	validation_1-merror:0.39498
[42]	validation_0-merror:0.27036	validation_1-merror:0.39355
[43]	validation_0-merror:0.26780	validation_1-merror:0.39245
[44]	validation_0-merror:0.26532	validation_1-merror:0.39104
[45]	validation_0-merror:0.26290	validation_1-merror:0.39036
[46]	validation_0-merror:0.26048	validation_1-merror:0.38971
[47]	validation_0-merror:0.25780	validation_1-merror:0.38933
[48]	validation_0-merror:0.25584	validation_1-merror:0.38826
[49]	validation_0-merror:0.25294	validation_1-merror:0.38743
[50]	validation_0-merror:0.25103	validation_1-merror:0.38655
[51]	validation_0-merror:0.24847	validation_1-merror:0.38630
[52]	validation_0-merror:0.24703	validation_1-merror:0.38553
[53]	validation_0-merror:0.24528	validation_1-merror:0.38510
[54]	validation_0-merror

[70]	validation_0-merror:0.21384	validation_1-merror:0.37945
[71]	validation_0-merror:0.21214	validation_1-merror:0.37879
[72]	validation_0-merror:0.21062	validation_1-merror:0.37859
[73]	validation_0-merror:0.20963	validation_1-merror:0.37857
[74]	validation_0-merror:0.20799	validation_1-merror:0.37832
[75]	validation_0-merror:0.20691	validation_1-merror:0.37845
[76]	validation_0-merror:0.20513	validation_1-merror:0.37828
[77]	validation_0-merror:0.20357	validation_1-merror:0.37759
[78]	validation_0-merror:0.20235	validation_1-merror:0.37691
[79]	validation_0-merror:0.20074	validation_1-merror:0.37726
[80]	validation_0-merror:0.19915	validation_1-merror:0.37671
[81]	validation_0-merror:0.19730	validation_1-merror:0.37604
[82]	validation_0-merror:0.19588	validation_1-merror:0.37577
[83]	validation_0-merror:0.19426	validation_1-merror:0.37595
[84]	validation_0-merror:0.19294	validation_1-merror:0.37553
[85]	validation_0-merror:0.19133	validation_1-merror:0.37510
[86]	validation_0-merror

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     eval_metric='merror', gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_con...
                                     num_class=19, num_parallel_tree=None,
                                     objective='multi:softmax',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=N

In [25]:
grid.best_params_

{'reg_lambda': 0.8}

In [ ]:
    "learning_rate": [0.05, 0.1, 0.2, 0.3, 0.4],
    "max_depth": [5, 6, 7],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.7, 0.8, 0.9, 1.0],
    'min_child_weight': [0.5, 1, 1.5],
    'gamma': [0, 0.1, 0.2],
    'reg_lambda': [0.8, 1, 1.5, 2]

## 输出特征重要性

In [ ]:
importance_dict = {col: importance for col, importance in zip(X_train.columns, xgb_clf.feature_importances_)}
sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

## 模型保存

In [ ]:
from sklearn.externals import joblib
import datetime
joblib.dump(xgb_clf, setting.BASE_DIR + "/data_model/model/xgboost_{}.pkl".format(datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d")))

# 模型评估

In [ ]:
y_pred = xgb_clf.predict(X_test)
y_pred.shape

In [ ]:
from sklearn.metrics import accuracy_score
y_true = y_test
accuracy_score(y_true, y_pred)

In [ ]:
for x in zip(y_true, y_pred):
    print(x)

In [ ]:
y_pred

## 加载官方测试集评估

In [ ]:
df_tsst = pd.read_csv(setting.BASE_DIR + "\data_model\data\sensor_test.csv")
df_tsst

In [ ]:
from xgboost_model.main import CreateFeature, XgboostModeling

obj = CreateFeature()
df_test = obj.load_test_sample()
df_test = obj.create_feature(df_test)
df_test = obj.derive_feature(df_test)

In [ ]:
df_test.shape

In [ ]:
df_test

In [ ]:
xgb_clf = joblib.load(setting.BASE_DIR + "/data_model/model/xgboost_{}.pkl".format(
    datetime.datetime.strftime(datetime.datetime.now(), "%Y-%m-%d")))

In [ ]:
X = df_test.values

In [ ]:
y_pred = xgb_clf.predict(X)
y_pred

In [ ]:
y_pred[:50]

In [ ]:
y_pred[50:100]

In [ ]:
y_pred[100:150]

In [ ]:
y_pred[150:200]

In [ ]:
y_pred[1000:1050]